In [ ]:
import numpy as np 
import pandas as pd 
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
df = pd.read_csv("/kaggle/input/global-warming-analysis/GlobalTemperatures.csv")

In [ ]:
df.head(3)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

In [ ]:
df.drop(columns=["LandAverageTemperatureUncertainty",
                 "LandMaxTemperatureUncertainty",
                 "LandMinTemperatureUncertainty",
                 "LandAndOceanAverageTemperatureUncertainty"],
        axis=1,inplace=True)

In [ ]:
df.head(4)

In [ ]:
def convert(y):
    y = (y * 1.8) + 32
    return float(y)

In [ ]:
df["LandAverageTemperature"]=df["LandAverageTemperature"].apply(convert)
df["LandMaxTemperature"]=df["LandMaxTemperature"].apply(convert)
df["LandMinTemperature"]=df["LandMinTemperature"].apply(convert)
df["LandAndOceanAverageTemperature"]=df["LandAndOceanAverageTemperature"].apply(convert)

In [ ]:
df["Date"] = pd.to_datetime(df["dt"])
df["Month"] = df["Date"].dt.month
df["Year"] = df["Date"].dt.year

In [ ]:
df.head(3)

In [ ]:
df.drop(["dt","Month"],axis=1,inplace=True)

In [ ]:
df = df[df.Year>=1850]
df = df.set_index(["Year"])

In [ ]:
df.head(3)

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

In [ ]:
df.describe()

In [ ]:
plt.figure(figsize=(10,7))
sns.heatmap(df.corr(),annot=True)

In [ ]:
sns.set_style("darkgrid")

In [ ]:
df.columns

In [ ]:
sns.kdeplot(df.LandAverageTemperature)

In [ ]:
sns.kdeplot(df.LandMinTemperature)

In [ ]:
sns.kdeplot(df.LandAndOceanAverageTemperature)

In [ ]:
sns.kdeplot(df.LandMaxTemperature)

In [ ]:
df.columns

In [ ]:
df = df[['LandAverageTemperature', 'LandMaxTemperature', 'LandMinTemperature',
       'LandAndOceanAverageTemperature']]

In [ ]:
x = df.drop("LandAndOceanAverageTemperature",axis=1)
y = df.LandAndOceanAverageTemperature

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,
                                test_size=0.25,
                                random_state=42)

In [ ]:
x_train.shape, x_test.shape

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor()
rfr.fit(x_train,y_train)

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.ensemble import RandomForestRegressor
forest = make_pipeline(
    SelectKBest(k="all"),
    StandardScaler(),
    RandomForestRegressor(
        n_estimators=100,
        max_depth=50,
        random_state=77,
        n_jobs=-1
    )
)
forest.fit(x_train, y_train)

In [ ]:
y_pred = forest.predict(x_test)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
err = abs(y_pred - y_test)
soln = 100 * (err/y_train)
accuracy = 100 - np.mean(soln)
print("Accuracy :", round(accuracy, 2), "%")